In [1]:
import pandas as pd

In [7]:
files = ["data/wiki_examples_flagged.csv", 'data/xav_training_examples.csv']
df = None
if len(files) > 1:
    temp = [0]*len(files)
    for i, file in enumerate(files):
        temp[i] = pd.read_csv(file)
    df = pd.concat(temp).reset_index()

In [8]:
df

,ID,Input,Output
0,1,"The Labrador Retriever, or simply Labrador, is...",developed
1,2,An angel is a ~_~celestial being in various tr...,"supernatural,religions,benevolent,religions"
2,3,"Attachment theory is a psychological, evolutio...","emotional,interactions"
3,4,The Gulag was the government ~_~administration...,"agency,convicts,convicts,convicts"
4,5,TreeHugger is a sustainability website that re...,"boasts,bought"
...,...,...,...
32,32,It would be nice to have a day at the beach. I...,great
33,33,We really needed more hands on deck in the lab...,"extremely,additional"
34,34,It's important that we get this project done o...,vital
35,35,"~_~First, I started off at the Population Neur...",Initially
